In [ ]:
import os
from datetime import datetime as dt

import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as L  # noqa: F401
import seaborn as sns
import torch
from pytorch_lightning.callbacks import (  # noqa: F401
    EarlyStopping,
    ModelSummary,
    TQDMProgressBar,
)
from pytorch_lightning.loggers import TensorBoardLogger  # noqa: F401
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm.notebook import tqdm
from xgboost import XGBRegressor

import hephaestus.single_row_models as sr

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Load and preprocess the train_dataset (assuming you have a CSV file)
df = pd.read_csv("./data/diamonds.csv")
df = df.rename(columns={"price": "target"})
# scale the non-target numerical columns
scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=[float, int]).columns
numeric_cols = numeric_cols.drop("target")
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
df.head()

,carat,cut,color,clarity,depth,table,target,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [19]:
single_row_config = sr.SingleRowConfig.generate(df, "target")
train_df, test_df = train_test_split(df.copy(), test_size=0.2, random_state=42)
train_dataset = sr.TabularDS(train_df, single_row_config)
test_dataset = sr.TabularDS(test_df, single_row_config)
model = sr.TabularRegressor(single_row_config, d_model=64, n_heads=4, lr=0.01)
model.predict_step(train_dataset[0:10])

tensor([[-0.1443],
        [-0.2108],
        [-0.1771],
        [-0.1494],
        [-0.1565],
        [-0.3035],
        [-0.2503],
        [-0.2996],
        [-0.1810],
        [-0.2901]])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1024 * 4,
    shuffle=True,
    collate_fn=sr.training.tabular_collate_fn,
    num_workers=7,
    persistent_workers=True,
)
val_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1024 * 4,
    collate_fn=sr.training.tabular_collate_fn,
    num_workers=7,
    persistent_workers=True,
)

In [12]:
logger_variant_name = "ScaleNumericLightningSmall"
logger_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
logger_name = f"{logger_time}_{logger_variant_name}"
print(f"Using logger name: {logger_name}")
logger = TensorBoardLogger(
    "runs",
    name=logger_name,
)
model_summary = ModelSummary(max_depth=3)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, mode="min")
progress_bar = TQDMProgressBar(leave=False)
trainer = L.Trainer(
    max_epochs=200,
    logger=logger,
    callbacks=[early_stopping, progress_bar, model_summary],
    log_every_n_steps=1,
)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Using logger name: 2025-03-16T12:51:05_ScaleNumericLightningSmall



   | Name                                       | Type                    | Params | Mode  | In sizes                             | Out sizes 
----------------------------------------------------------------------------------------------------------------------------------------------------
0  | model                                      | TabularEncoderRegressor | 148 K  | train | [[3, 6], [3, 3]]                     | [3, 1]    
1  | model.tabular_encoder                      | TabularEncoder          | 136 K  | train | [[3, 6], [3, 3]]                     | [3, 9, 64]
2  | model.tabular_encoder.embeddings           | Embedding               | 2.2 K  | train | [3, 9]                               | [3, 9, 64]
3  | model.tabular_encoder.transformer_encoder1 | TransformerEncoderLayer | 33.5 K | train | [[3, 9, 64], [3, 9, 64], [3, 9, 64]] | [3, 9, 64]
4  | model.tabular_encoder.transformer_encoder2 | TransformerEncoderLayer | 33.5 K | train | [[3, 9, 64], [3, 9, 64], [3, 9, 64]] | [3,

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
y_hat = model.predict_step(train_dataset[0:64])
y = train_dataset[0:64].target
print(y.shape, y_hat.shape)
print(mean_squared_error(y, y_hat))

torch.Size([64]) torch.Size([64, 1])
47270.78043757149


In [14]:
y_hat

tensor([[ 485.6536],
        [ 573.3076],
        [ 932.0192],
        [ 513.4769],
        [ 518.9960],
        [ 493.0768],
        [ 504.0769],
        [ 483.8365],
        [1003.1820],
        [ 529.4319],
        [ 501.5361],
        [ 494.0701],
        [ 544.2435],
        [ 499.4337],
        [ 604.9996],
        [ 606.1221],
        [ 484.1013],
        [ 494.8521],
        [ 499.3753],
        [ 506.7441],
        [ 501.3183],
        [ 639.2293],
        [ 492.0422],
        [ 516.8468],
        [ 515.5181],
        [ 549.5970],
        [ 503.8080],
        [ 495.5121],
        [ 605.7529],
        [ 503.0171],
        [ 498.6073],
        [ 497.3055],
        [ 565.1989],
        [ 546.0236],
        [ 553.9990],
        [ 510.8453],
        [ 692.1205],
        [ 595.3718],
        [ 582.5145],
        [ 488.3560],
        [ 486.6051],
        [ 501.8453],
        [1037.5969],
        [ 736.6771],
        [ 504.5949],
        [ 588.7340],
        [ 483.9549],
        [ 517

In [17]:
1 / 0  # Stop here to avoid running the rest of the code

ZeroDivisionError: division by zero

In [ ]:
model = sr.TabTransformer(train_dataset, n_heads=8).to(train_dataset.device)

batch_size = 3
test_num = train_dataset.X_train_numeric[0:batch_size, :]
test_num_mask = sr.mask_tensor(test_num, model)
test_cat = train_dataset.X_test_categorical[0:batch_size, :]
test_cat_mask = sr.mask_tensor(test_cat, model)
with torch.no_grad():
    x = model(
        test_num_mask,
        test_cat_mask,
        task="mlm",
    )
x[0].shape, x[1].shape

In [ ]:
# Masked Tabular Modeling
base_model_name = "is_model_global2"

model_time = dt.now()
model_time = model_time.strftime("%Y-%m-%dT%H:%M:%S")
model_name = f"{base_model_name}_{model_time}"

model_save_path = "./checkpoints/mtm_models_small.pt"

In [ ]:
model_list = os.listdir("./checkpoints")
if model_save_path.split("/")[-1] in model_list:
    print("Model already exists")
    model_exists = True
else:
    print("Model does not exist")
    model_exists = False

if model_exists:
    model.load_state_dict(torch.load(model_save_path))
else:
    sr.mtm(model, train_dataset, model_name, epochs=100, batch_size=1000, lr=0.001)
    torch.save(model.state_dict(), model_save_path)

In [ ]:
# regression_performance = sr.fine_tune_model(
#     model, train_dataset, model_name="FT100", n_rows=100, epochs=100
# )
# regression_performance

In [ ]:
n_train_rows = [
    # 10,
    100,
    1_000,
    2_000,
    5_000,
    10_000,
    15_000,
    30_000,
    # 40_000,
    train_dataset.X_train.shape[0],
]

In [ ]:
def train_multiple_sizes(pt_model_path, train_dataset, n_train_rows, n_epochs=100):
    model = sr.TabTransformer(train_dataset, n_heads=8).to(train_dataset.device)
    if pt_model_path is not None:
        model.load_state_dict(torch.load(pt_model_path))

    regression_performance = sr.fine_tune_model(
        model,
        train_dataset,
        model_name=f"ft_{n_train_rows}",
        n_rows=n_train_rows,
        epochs=n_epochs,
    )

    return regression_performance

In [ ]:
hephaestus_results_no_pre_train = []
pbar = tqdm(n_train_rows)
for i in pbar:
    pbar.set_description(f"n_rows: {i}")
    loss = train_multiple_sizes(None, train_dataset, i, n_epochs=250)
    hephaestus_results_no_pre_train.append(loss)

In [ ]:
no_pt_df = pd.DataFrame(hephaestus_results_no_pre_train)
no_pt_df["model"] = "Hephaestus No Fine Tune"
no_pt_df

In [ ]:
hephaestus_results = []
pbar = tqdm(n_train_rows)
for i in pbar:
    pbar.set_description(f"n_rows: {i}")
    loss = train_multiple_sizes(model_save_path, train_dataset, i, n_epochs=250)
    hephaestus_results.append(loss)

In [ ]:
hephaestus_df = pd.DataFrame(hephaestus_results)
hephaestus_df["model"] = "Hephaestus"
hephaestus_df

In [ ]:
hephaestus_df.loc[hephaestus_df.n_rows == 1000, "test_loss"].values

In [ ]:
diamonds_data = pd.read_csv("./data/diamonds.csv")

# Encode categorical features using LabelEncoder
label_encoders = {}
categorical_features = ["cut", "color", "clarity"]
for feature in categorical_features:
    le = LabelEncoder()
    diamonds_data[feature] = le.fit_transform(diamonds_data[feature])
    label_encoders[feature] = le

# Split the train_dataset into features (X) and target (y)
X = diamonds_data.drop("price", axis=1)
y = diamonds_data["price"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize and train the XGBoost regressor
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_regressor.fit(
    X_train[0:batch_size],
    y_train[0:batch_size],
)

# Predict on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:,.2f}")

# You can also access feature importance scores
# feature_importances = xgb_regressor.feature_importances_
# print("Feature Importance:")
# for feature, importance in zip(X.columns, feature_importances):
#     print(f"{feature}: {importance:.4f}")

In [ ]:
def xgb_tester(train_set_size):
    xgb_regressor = XGBRegressor(n_estimators=120, learning_rate=0.1, random_state=42)
    xgb_regressor.fit(
        X_train[0:train_set_size],
        y_train[0:train_set_size],
    )

    y_pred = xgb_regressor.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return {"n_rows": train_set_size, "test_loss": mse}


xgb_losses = []
for i in tqdm(n_train_rows):
    mse = xgb_tester(i)
    xgb_losses.append(mse)

In [ ]:
xgb_df = pd.DataFrame(xgb_losses)
xgb_df["model"] = "XGBoost"

xgb_df

In [ ]:
loss_df = pd.concat([hephaestus_df, xgb_df, no_pt_df])  # , no_pt_df
loss_df = loss_df.loc[loss_df["n_rows"] != 10]
loss_df.sample(10)

In [ ]:
# Define the colors for each model
# colors = {"Hephaestus": "blue", "XGBoost": "red"}

# Create a figure and axis object
fig, ax = plt.subplots()

# Loop through each model and plot the test loss as a line
for model, group in loss_df.groupby("model"):
    ax.plot(group["n_rows"], group["test_loss"], label=model)

# Set the axis labels and legend
ax.set_xlabel("Number of Rows")
ax.set_ylabel("Test Loss")
ax.legend()
# set x axis to log scale
ax.set_xscale("log")

# Show the plot
plt.show()

In [ ]:
# Define the colors for each model
# colors = {"Hephaestus": "blue", "XGBoost": "red"}

# Create a figure and axis object
fig, ax = plt.subplots()

# Loop through each model and plot the test loss as a line
for model, group in loss_df.loc[loss_df["model"] != "Hephaestus No Fine Tune"].groupby(
    "model"
):
    ax.plot(group["n_rows"], group["test_loss"], label=model)

# Set the axis labels and legend
ax.set_xlabel("Number of Rows")
ax.set_ylabel("Test Loss")
ax.legend()
# set x axis to log scale
ax.set_xscale("log")

# Show the plot
plt.show()

In [ ]:
# Spread the data to have columns for the loss of each model
# loss_df =
loss_percent_df = loss_df.pivot(
    index="n_rows", columns="model", values="test_loss"
).reset_index()
loss_percent_df["percent_improvement"] = (
    loss_percent_df["XGBoost"] - loss_percent_df["Hephaestus"]
) / loss_percent_df["XGBoost"]

In [ ]:
loss_percent_df

In [ ]:
ax = sns.lineplot(data=loss_percent_df, x="n_rows", y="percent_improvement")
plt.axhline(y=0, color="black", linestyle="--")
ax.set_yticks(loss_percent_df["percent_improvement"].round(2))
ax.set_xticks(loss_percent_df["n_rows"])
# ax.set_xscale("log")
# X lables at 45 degree angle
plt.xticks(rotation=45)
# plt.xlabel